In [1]:
!pip install google-cloud-bigquery pandas
import os

In [ ]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] ="/home/gaz/Descargas/starry-aegis-443515-b9-2bc75a15e73d.json"


In [7]:
import pandas as pd
from google.cloud import bigquery
from google.cloud import storage
import datetime

# Inicializar BigQuery y Google Storage
bq_client = bigquery.Client()
storage_client = storage.Client()

# SE definen coonstantes
PROJECT_ID = 'starry-aegis-443515-b9'
DATASET_ID = 'insight_hub'
STAGING_TABLE = f'{PROJECT_ID}.{DATASET_ID}.staging_table'
TRANSFORMED_TABLE = f'{PROJECT_ID}.{DATASET_ID}.transformed_table'
BUCKET_NAME = 'insight_hub'
CSV_FILE_NAME = f'transformed_data_{datetime.datetime.now().strftime("%Y%m%d")}.csv'

# Extraccion desde dataset
def extract_data():
    query = f"""
    SELECT *
    FROM `{PROJECT_ID}.{DATASET_ID}.your_existing_table`
    WHERE your_conditions_to_filter_new_data
    """
    df = bq_client.query(query).to_dataframe()
    return df

# Carga de datos en staging
def load_data_to_staging(df):
    job_config = bigquery.LoadJobConfig(
        write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
    )
    job = bq_client.load_table_from_dataframe(df, STAGING_TABLE, job_config=job_config)
    job.result()  # Wait for the job to complete

# TRansformacion de la data
def transform_data():
    query = f"""
    CREATE OR REPLACE TABLE `{TRANSFORMED_TABLE}` AS
    SELECT
        VendorNumber,
        VendorName,
        SalesQuantity,
        -- Add your transformation logic here
    FROM `{STAGING_TABLE}`
    """
    bq_client.query(query).result()  # Execute the transformation query

# Exporto a CSV
def export_to_csv():
    destination_uri = f'gs://{BUCKET_NAME}/{CSV_FILE_NAME}'
    query = f"""
    SELECT *
    FROM `{TRANSFORMED_TABLE}`
    """
    extract_job = bq_client.extract_table(
        f'{TRANSFORMED_TABLE}',
        destination_uri,
        location='US'  # Change if your dataset is in a different location
    )
    extract_job.result()  # Wait for the job to complete
    print(f'Exported data to {destination_uri}')

# Se corre ELT
def main():
    df = extract_data()
    load_data_to_staging(df)
    transform_data()
    export_to_csv()

if __name__ == '__main__':
    main()

NotFound: 404 Not found: Table starry-aegis-443515-b9:insight_hub.your_existing_table was not found in location US; reason: notFound, message: Not found: Table starry-aegis-443515-b9:insight_hub.your_existing_table was not found in location US

Location: US
Job ID: 9aa3122f-5f9a-4a5e-89c6-07f889f680fd
